In [1]:
#!pip install requests

Возможные улучления
- списки валидации
- выбор даты
- обязательность полей
- дизайн

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

import requests

import pandas as pd
import numpy as np


In [3]:
URL = 'http://127.0.0.1:5000'

In [4]:
parameters = \
    {
        'st_code_snd':['', 'str', 'happy:)pgkwagon:):)', 'станция отправления'],
        'st_code_rsv':['', 'str', 'pgk2022newyear:)pgk', 'станция назначения'],
        'date_depart_year':[0, 'int64', '2022', 'год'],
        'date_depart_month':[0, 'int64', '9', 'месяц'],
        'date_depart_week':[0, 'int64', '35', 'неделя'],
        'date_depart_day':[0, 'int64', '1', 'день'],
        'date_depart_hour':[0, 'int64', '9', 'час'],
        'fr_id':[0, 'float64', '2261.0', 'id груза'],
        'route_type':[0, 'float64', '3.0', 'тип отправки (1,3 или 4)'],
        'is_load':[0, 'int64', '0', 'признак гружёности (1 - гружёный, 0 - порожний)'],
        'rod':[0, 'int64', '2', 'род подвижного состава'],
        'common_ch':[0, 'float64', '10.0', 'обобщённая характеристики вагона'],
        'vidsobst':[0, 'float64', '102.0', 'вид собственности'],
        'distance':[0, 'float64', '2930.0', 'расстояние рейса'],
        'snd_org_id':[0, 'int64', '2974', 'грузоотправитель'],
        'rsv_org_id':[0, 'int64', '2847', 'грузополучатель'],
        'snd_roadid':[0, 'int32', '28', 'дорога отправления'],
        'rsv_roadid':[0, 'int32', '28', 'дороги назначения'],
        'snd_dp_id':[0, 'int64', '134', 'регион отправления'], 
        'rsv_dp_id':[0, 'int64', '125', 'регион назначения']}
data = {}

In [5]:
app = dash.Dash(__name__)
server = app.server

In [6]:
children = [
        html.H1(children="Сервис для прогнозирования длительности движения вагонов",
               style={'color':'#8A2432', 'text-align': 'center'}),
        html.Hr()]

for n in parameters.keys():
    children.append(html.Div(children=[\
            dcc.Input(id='input-'+n, value=parameters[n][2], placeholder='<Введите значение>', type='text', \
                       style={'width': '25%', 'border-style': 'solid', 'border-width': '1px', \
                              'border-color': 'gray', 'margin': '0', 'margin-left': '25px', \
                              'color':'#FF7313', 'display': 'inline', 'padding': '5px'}),                       
            html.P(children=f' - "{parameters[n][3]}"', style={'margin-left': '25px', 'color':'#8A2432', \
                        'padding':'0', 'display': 'inline'})
            
                                      ]))

        
children.append(html.Div(children=[
    html.Br(),
    html.Br(),
    html.Button('Обработать', id='submit-val', 
                style={'background': '#FF7313', 'color': 'white', 'border-style': 'outset', \
                       'border-color': '#8A2432', 'height': '30px', 'width': '300px', \
                       'font': 'bold15px arial,sans-serif', 'text-shadow': 'none', \
                      'margin-left': '25px'}),
    html.Br(),
    html.Br(),
    html.Br(),
    html.P(children='Оценка длительности:', style={'margin-left': '25px', 'color':'#8A2432'}),
    html.Div(id='container-button-basic', children='Нажмите "Обработать" для получения результата', \
            style={'margin-left': '25px', 'color':'#8A2432', 'font-weight': 'bold'})
]));

app.layout = html.Div(children=children, style={'background':'#C6C6C6', 'padding': '10px', \
           'border':'3px solid #8A2432', 'width': '80%', 'margin': 'auto'})

In [7]:
@app.callback(
    Output('container-button-basic', 'children'),
    Input('submit-val', 'n_clicks'),
    [State('input-st_code_snd', 'value'), \
     State('input-st_code_rsv', 'value'), \
     State('input-date_depart_year', 'value'), \
     State('input-date_depart_month', 'value'), \
     State('input-date_depart_week', 'value'), \
     State('input-date_depart_day', 'value'), \
     State('input-date_depart_hour', 'value'), \
     State('input-fr_id', 'value'), \
     State('input-is_load', 'value'), \
     State('input-rod', 'value'), \
     State('input-common_ch', 'value'), \
     State('input-vidsobst', 'value'), \
     State('input-distance', 'value'), \
     State('input-snd_org_id', 'value'), \
     State('input-rsv_org_id', 'value'), \
     State('input-snd_roadid', 'value'), \
     State('input-rsv_roadid', 'value'), \
     State('input-snd_dp_id', 'value'), \
     State('input-rsv_dp_id', 'value') \
    ]
)
def predict(n_clicks, st_code_snd, st_code_rsv, date_depart_year, date_depart_month,
              date_depart_week, date_depart_day, date_depart_hour, fr_id, is_load, 
              rod, common_ch, vidsobst, distance, snd_org_id, rsv_org_id, snd_roadid, 
              rsv_roadid, snd_dp_id, rsv_dp_id):

    params = {}
    
    if st_code_snd is not None:
        params['st_code_snd'] = st_code_snd
        
    if st_code_rsv is not None:
        params['st_code_rsv'] = st_code_rsv

    if date_depart_year is not None:
        params['date_depart_year'] = date_depart_year

    if date_depart_month is not None:
        params['date_depart_month'] = date_depart_month

    if date_depart_week is not None:
        params['date_depart_week'] = date_depart_week

    if date_depart_day is not None:
        params['date_depart_day'] = date_depart_day

    if date_depart_hour is not None: 
        params['date_depart_hour'] = date_depart_hour

    if fr_id is not None:
        params['fr_id'] = fr_id

    if is_load is not None: 
        params['is_load'] = is_load

    if rod is not None:
        params['rod'] = rod

    if common_ch is not None: 
        params['common_ch'] = common_ch

    if vidsobst is not None:
        params['vidsobst'] = vidsobst

    if distance is not None:
        params['distance'] = distance

    if snd_org_id is not None: 
        params['snd_org_id'] = snd_org_id

    if rsv_org_id is not None:
        params['rsv_org_id'] = rsv_org_id

    if rsv_org_id is not None:
        params['rsv_org_id'] = rsv_org_id

    if rsv_org_id is not None:
        params['rsv_org_id'] = rsv_org_id

    if rsv_org_id is not None:
        params['rsv_org_id'] = rsv_org_id

    if rsv_org_id is not None:
        params['rsv_org_id'] = rsv_org_id

    responce = requests.get(f'{URL}/predict_delivery_time', params=params)
    result = responce.text
    
    return 'Результат = {}'.format(result)

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('127.0.0.1', 8051, server)
    
# app.run_server(debug=True, host = '127.0.0.1')

 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2022 13:12:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 13:12:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 13:12:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 13:12:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 14:04:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 14:04:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 14:04:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 14:04:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 15:13:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 15:13:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 15:13:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 15:32:58] "POST /_dash-update-component HTTP/1.1" 200 -
